# RGI10 (Asia North)

F. Maussion & S. Galos, June-December 2021

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist, open_zip_shapefile
import os

## Files and storage paths

In [ ]:
# Region of interest
reg = 10

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# RGI v6 file for comparison later 
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', '10_rgi60_NorthAsia.zip')

In [ ]:
# Specific to this region: boxes where data has to be selected differently
support_dir = os.path.join(data_dir, 'l0_support_data')

# OK path to file
box_file = os.path.join(support_dir, 'rgi10_boxes.zip')

### Load the input data

In [ ]:
# Read L2 files
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

### List of submissions 

In [ ]:
sdf, df_cat = submission_summary(shp)
sdf

- 636 is RGI6
- 698 is GAMDAMv2 - we use it
- 726 is a mapping of a few remaining nominal glaciers on three De Long Islands
- 743 is an update of the Barr inventory for Kamchatka

In [ ]:
# # Optional: write out selection in intermediate shape files for manual GIS review
# tmp_output_dir = mkdir(os.path.join(data_dir, 'l0_tmp_data', f'rgi{reg:02d}_inventories'))
# tmp_output_dir_tar = mkdir(os.path.join(data_dir, 'l0_tmp_data'))
# for subid in shp.subm_id.unique():
#     s_loc = shp.loc[shp.subm_id == subid]
#     s_loc.to_file(tmp_output_dir + f'/subm_{int(subid):03d}.shp')
# print('Taring...')
# print(subprocess.run(['tar', '-zcvf', f'{tmp_output_dir_tar}/rgi{reg:02d}_inventories.tar.gz', '-C', 
#                       os.path.join(data_dir, 'l0_tmp_data'), f'rgi{reg:02d}_inventories']))

## Outline selection 

In [ ]:
glims_rgi = shp.loc[shp.subm_id.isin([636])].copy()
glims_rgi['is_rgi6'] = True
all_others = shp.loc[shp.subm_id.isin([698, 726, 743])].copy()
all_others['is_rgi6'] = False

In [ ]:
# Preselected areas to remove
box = open_zip_shapefile(support_dir + '/rgi10_boxes.zip')

In [ ]:
# Remove the new regions from rgi
rp = glims_rgi.representative_point()
rp = rp.to_frame('geometry')
rp['orig_index'] = glims_rgi.index
difference = gpd.overlay(rp, box, how='difference')
glims_rgi = glims_rgi.loc[difference['orig_index']].copy()

In [ ]:
# Size filter?
needs_size_filter(glims_rgi), needs_size_filter(all_others)

In [ ]:
print(len(all_others))
all_others = size_filter(all_others)
print(len(all_others))

In [ ]:
rgi7 = pd.concat([glims_rgi, all_others])

### Some sanity checks 

In [ ]:
sdf, df_class = submission_summary(rgi7)
df_class

In [ ]:
# Check the orphaned rock outcrops
orphan_f = os.path.join(data_dir, 'l1_orphan_interiors', f'RGI{reg:02d}', f'RGI{reg:02d}.shp')
if os.path.exists(orphan_f):
    orphan_f = gpd.read_file(orphan_f)
    check = np.isin(rgi7.subm_id.unique(), orphan_f.subm_id.unique())
    if np.any(check):
        print(f'Orphan rock outcrops detected in subm_id {rgi7.subm_id.unique()[check]}')
        orphan_f['area'] = orphan_f.to_crs({'proj':'cea'}).area

### Plots 

In [ ]:
plot_map(rgi7, reg, figsize=(22, 10), linewidth=3, loc='upper center')

In [ ]:
plot_map(rgi7, reg, figsize=(22, 10), linewidth=3, loc='upper center', is_rgi6=True)

In [ ]:
plot_date_hist(rgi7, reg)

### Text for github

In [ ]:
fgh = sdf.T
fgh

In [ ]:
print(fgh.to_markdown(headers=np.append(['subm_id'], fgh.columns)))

## Write out and tar 

In [ ]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
rgi7.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))

## Consistency check with RGI6 

In [ ]:
rgi6 = open_zip_shapefile(rgi6_reg_file)

In [ ]:
len(rgi7), len(rgi6)

Test the areas:

In [ ]:
rgi6['area'] = rgi6.to_crs({'proj':'cea'}).area 

In [ ]:
print('Area RGI7a (km2)', rgi7['area'].sum() * 1e-6)
print('Area RGI6 (km2)', rgi6['area'].sum() * 1e-6)
print('diff areas RGI6 - RGI7 computed by us (km2)', (rgi6['area'].sum() - rgi7['area'].sum()) * 1e-6)

In [ ]:
# Remove the ids
rp = rgi6.representative_point()
rp = rp.to_frame('geometry')
rp['orig_index'] = rgi6.index

difference = gpd.overlay(rp, box, how='difference')
rgi6_old = rgi6.loc[difference['orig_index']].copy()
difference = gpd.overlay(rp, box, how='intersection')
rgi6_new = rgi6.loc[difference['orig_index']].copy()
assert len(rgi6_new) + len(rgi6_old) == len(rgi6)

In [ ]:
print(f'N1 = {len(rgi6_old)} , N2 = {len(glims_rgi)}')
print('Area RGI7 (km2)', glims_rgi['area'].sum() * 1e-6)
print('Area RGI6 (km2)', rgi6_old['area'].sum() * 1e-6)
print('diff', (rgi6_old['area'].sum() - glims_rgi['area'].sum()) * 1e-6)